In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import poisson 
import math
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn import preprocessing
from pretty_html_table import build_table

def sum_of_lower_upper(mat):
    r = len(mat)
    c = len(mat[0])
    diag = np.trace(mat)
    lower_sum = 0
    upper_sum = 0
    over_sum = 0
    under_sum = 0
    gg_sum = 0
    ng_sum = 0 

    for i in range(r):
        for j in range(c):
            if (i<j):
                upper_sum += mat[i][j]

    for i in range(r):
        for j in range(c):
            if (j<i):
                lower_sum += mat[i][j]
    
    for i in range(r):
        for j in range(c):
            if (i+j>=3):
                over_sum += mat[i][j]
            else:
                under_sum += mat[i][j]
    
    for i in range(r):
        for j in range(c):
            if (i==0) or (j==0):
                ng_sum += mat[i][j]
            else:
                gg_sum += mat[i][j]

    return (upper_sum, lower_sum, diag, over_sum, under_sum, ng_sum, gg_sum)


## Gathering Lamda from season long performance

In [2]:
df1 = pd.read_csv('GatheredData/Full20xxFiles/Full2017.csv')
df2 = pd.read_csv('GatheredData/Full20xxFiles/Full2018.csv')
df3 = pd.read_csv('GatheredData/Full20xxFiles/Full2019.csv')
df4 = pd.read_csv('GatheredData/Full20xxFiles/Full2020.csv')
df5 = pd.read_csv('GatheredData/Full20xxFiles/Full2021.csv')
df6 = pd.read_csv('GatheredData/Full20xxFiles/Full2022.csv')
PRED = pd.read_csv('UpcomingFixtures/Prediction.csv')
pred = PRED.copy()
#Keeping bookmakers odds
Odds = PRED.copy()
Odds = PRED[['HTeam', 'ATeam','B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5', 'GG', 'NG']]

df1 = df1[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df2 = df2[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df3 = df3[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df3.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df3.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df4 = df4[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df4.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df4.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df5 = df5[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df5.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df5.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df6 = df6[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]

df1.rename(columns = {'Date_x':'Date'}, inplace = True)
df2.rename(columns = {'Date_x':'Date'}, inplace = True)
df3.rename(columns = {'Date_x':'Date'}, inplace = True)
df4.rename(columns = {'Date_x':'Date'}, inplace = True)
df5.rename(columns = {'Date_x':'Date'}, inplace = True)


Season = pd.read_csv('GatheredData/Full20xxFiles/Full2023.csv')
Season = Season[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
Season = Season.iloc[38:]


DF = pd.concat([df1, df2, df3, df4, df5, df6, Season], ignore_index=True)


DF = DF[ DF['HM'] != 0] 
DF = DF[ DF['HM'] != 1] 
DF = DF[ DF['HM'] != 2] 
DF = DF[ DF['HM'] != 3]
subset = DF[['Date', 'HTeam', 'ATeam']].copy()

extra_stats = DF.copy()
extra_pred = pred.copy()

DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']
pred['PTSDIF/M']=pred['HPTS/M']-pred['APTS/M']
DF = DF[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
pred = pred[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
cut = DF.shape[0]
print(cut)
DF = pd.concat([DF, pred], ignore_index=True)
try:
    DF['HxG'] = DF['HxG'].div(DF['HM'])
    DF['HG'] = DF['HG'].div(DF['HM'])
    DF['HGA'] = DF['HGA'].div(DF['HM'])
    DF['HxGA'] = DF['HxGA'].div(DF['HM'])
    DF['AxG'] = DF['AxG'].div(DF['AM'])
    DF['AG'] = DF['AG'].div(DF['AM'])
    DF['AxGA'] = DF['AxG'].div(DF['AM'])
    DF['AGA'] = DF['AGA'].div(DF['AM'])
    pred['HxG'] = pred['HxG'].div(pred['HM'])
    pred['HG'] = pred['HG'].div(pred['HM'])
    pred['HGA'] = pred['HGA'].div(pred['HM'])
    pred['HxGA'] = pred['HxGA'].div(pred['HM'])
    pred['AxG'] = pred['AxG'].div(pred['AM'])
    pred['AG'] = pred['AG'].div(pred['AM'])
    pred['AxGA'] = pred['AxG'].div(pred['AM'])
    pred['AGA'] = pred['AGA'].div(pred['AM'])

except ZeroDivisionError:
    DF['HxG'] = 0
    DF['HG'] = 0
    DF['HGA'] = 0
    DF['HxGA'] = 0
    DF['AxG'] = 0
    DF['AG'] = 0
    DF['AxGA'] = 0
    DF['AGA'] = 0

HGoals = []
AGoals = []
    
for i, x in DF.iloc[:cut].iterrows():
    a=x['Result'].split('\'')
    HGoals.append(a[1])
    AGoals.append(a[3])
    
DF=np.array(DF)
DFnorm = preprocessing.normalize(DF[:, 0:21], axis=0)
pred = DFnorm[cut:]
DF = DFnorm[:cut]

x=DF


y1=np.array(HGoals)
y2=np.array(AGoals)

reg1 = LinearRegression().fit(x, y1)
reg2 = LinearRegression().fit(x, y2)

LamdaHome= reg1.predict(pred)
LamdaAway= reg2.predict(pred)

2071


## Gathering Lamda from 2 month form performance

In [3]:
df1 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2017Monthly.csv')
df2 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2018Monthly.csv')
df3 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2019Monthly.csv')
df4 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2020Monthly.csv')
df5 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2021Monthly.csv')
df6 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2022Monthly.csv')
pred = PRED.copy()
#Keeping bookmakers odds
Odds = PRED.copy()
Odds = PRED[['HTeam', 'ATeam','B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5', 'GG', 'NG']]

df1 = df1[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df2 = df2[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df3 = df3[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df3.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df3.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)
df4 = df4[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df4.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df4.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)
df5 = df5[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df5.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df5.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)
df6 = df6[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df6.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df6.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)

SeasonForm = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2023Monthly.csv')


try:
    SeasonForm = SeasonForm[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
except KeyError:
    SeasonForm = SeasonForm[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
    SeasonForm.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
    SeasonForm.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)
    
SeasonForm = SeasonForm.iloc[38:]

DF = pd.concat([df1, df2, df3, df4, df5, df6, SeasonForm], ignore_index=True)
merged = pd.merge(subset, DF, on=['Date', 'HTeam', 'ATeam'], how='left')
print(subset.iloc[1361])
print(DF.iloc[1361])
print(merged[merged.isna().any(axis=1)])

# resulting dataframe will have columns of B where the triplet of A['a', 'b', 'c'] exists in B
DF = merged[DF.columns]

DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']
pred['PTSDIF/M']=pred['HPTS/M']-pred['APTS/M']
DF = DF[['HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
pred = pred[['HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
cut = DF.shape[0]
DF = pd.concat([DF, pred], ignore_index=True)



try:
    DF['HxG'] = DF['HxG'].div(DF['HM']).fillna(0)
    DF['HG'] = DF['HG'].div(DF['HM']).fillna(0)
    DF['HGA'] = DF['HGA'].div(DF['HM']).fillna(0)
    DF['HxGA'] = DF['HxGA'].div(DF['HM']).fillna(0)
    DF['AxG'] = DF['AxG'].div(DF['AM']).fillna(0)
    DF['AG'] = DF['AG'].div(DF['AM']).fillna(0)
    DF['AxGA'] = DF['AxG'].div(DF['AM']).fillna(0)
    DF['AGA'] = DF['AGA'].div(DF['AM']).fillna(0)
    pred['HxG'] = pred['HxG'].div(pred['HM']).fillna(0)
    pred['HG'] = pred['HG'].div(pred['HM']).fillna(0)
    pred['HGA'] = pred['HGA'].div(pred['HM']).fillna(0)
    pred['HxGA'] = pred['HxGA'].div(pred['HM']).fillna(0)
    pred['AxG'] = pred['AxG'].div(pred['AM']).fillna(0)
    pred['AG'] = pred['AG'].div(pred['AM']).fillna(0)
    pred['AxGA'] = pred['AxG'].div(pred['AM']).fillna(0)
    pred['AGA'] = pred['AGA'].div(pred['AM']).fillna(0)

except ZeroDivisionError:
    DF['HxG'] = 0
    DF['HG'] = 0
    DF['HGA'] = 0
    DF['HxGA'] = 0
    DF['AxG'] = 0
    DF['AG'] = 0
    DF['AxGA'] = 0
    DF['AGA'] = 0


print(DF.iloc[:cut])
        
HGoals = []
AGoals = []
for i, x in DF.iloc[:cut].iterrows():
    a=str(x['Result']).split('\'')
    try:
        HGoals.append(a[1])
        AGoals.append(a[3])
    except IndexError:
        print(x)
        print(DF.iloc[:cut].iloc[i-1])

DF=np.array(DF)

DFnorm = preprocessing.normalize(DF[:, 0:21].astype('float64'), axis=0)
pred = DFnorm[cut:]
DF = DFnorm[:cut]

x=DF

y1=np.array(HGoals)
y2=np.array(AGoals)

reg1 = LinearRegression().fit(x, y1)
reg2 = LinearRegression().fit(x, y2)

LamdaHomeForm= reg1.predict(pred)
LamdaAwayForm= reg2.predict(pred)

Date           17/09/2021
HTeam    Newcastle United
ATeam               Leeds
Name: 1560, dtype: object
Date                     07/02/2021
HxG                           12.05
HM                               12
HTeam       Wolverhampton Wanderers
ATeam                     Leicester
HG                               12
HGA                              20
HxGA                          15.48
HDC                        3.166667
HODC                       7.083333
HPPDA                         15.93
HOPPDA                        11.56
HPTS/M                         0.75
AM                               12
AG                               20
AGA                              11
AxG                           16.59
AxGA                          10.23
ADC                             7.0
AODC                       4.333333
APPDA                           9.7
AOPPDA                        12.24
APTS/M                          2.0
Result                 ['0 ', ' 0']
B365H                           

/tmp/ipykernel_12288/2412402004.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']


In [4]:
LamdaAwayFinal = [(LamdaAway[i] * 0.4 + LamdaAwayForm[i] * 0.6) for i in range(len(LamdaAway))]
LamdaHomeFinal = [(LamdaHome[i] * 0.4 + LamdaHomeForm[i] * 0.6) for i in range(len(LamdaHome))]

print(LamdaHomeFinal)
print(LamdaAwayFinal)

[0.9735857988948937, 0.9382631291562095, 1.9512134900845013, 1.850682184949809, 1.8933537724298106, 1.4196921571288232, 1.111409655975312, 1.954774834905799, 1.2498629299455803, 1.7881056499876005, 2.242998849206133, 1.6500179955549688, 2.0258051193831936, 2.430351112382981, 2.1202948196221096, 1.4947391284186446, 1.840832486472853, 0.7699488871493623, 1.8827594167494444, 2.563214530997052]
[2.179986378576892, 1.8308642559215618, 0.962197552011479, 1.0006501504101284, 1.4061895166858847, 1.1208046978403972, 2.0898560968709323, 1.6466063797685564, 1.974875734082525, 1.6895130002574634, 1.169027215033718, 1.7706227239510388, 0.9126764573728704, 1.2635199621059658, 1.0496263476631038, 1.1920841027876505, 1.7461932678063494, 2.0851142563900824, 1.2863316631083177, 1.1388915880235846]


In [5]:
k = range(12)
PH = np.zeros((len(k), 1))
PA = np.zeros((1, len(k)))
PRED['P(Hwin)']= np.nan
PRED['P(Awin)']= np.nan
PRED['P(Draw)']= np.nan
PRED['P(Over)']= np.nan
PRED['P(Under)']= np.nan
PRED['P(GG)']=np.nan
PRED['P(NG)']=np.nan

Pie_list = []
for j, l in enumerate(LamdaHome):
    for i in k:
        PH[i, 0] = (math.exp(-l)*l**k[i]) /math.factorial(k[i])
        PA[0, i] = (math.exp(-LamdaAway[j])*LamdaAway[j]**k[i]) /math.factorial(k[i])
    Pscore = np.matmul(PH, PA) 
    df = pd.DataFrame(Pscore, columns = ['0','1','2', '3', '4', '5', '6', '7', '8', '9', '10', '11'])
    PRED.loc[j,'P(Awin)'], PRED.loc[j,'P(Hwin)'], PRED.loc[j,'P(Draw)'], PRED.loc[j,'P(Over)'], PRED.loc[j,'P(Under)'], PRED.loc[j,'P(NG)'], PRED.loc[j,'P(GG)']  = sum_of_lower_upper(Pscore)
    PRED['P(Over)'] = 1 - PRED['P(Under)']
    df = df.iloc[0:5, 0:5]
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    
    G = []
    for i in range(0, 5):
        for j in range(0, 5):
            G.append(df.loc[i][j])

            
    Pie_Df = pd.DataFrame(G, columns = ['P(score)'])
    Pie_Df['score']= ['0 - 0',
                      '0 - 1',
                      '0 - 2',
                      '0 - 3',
                      '0 - 4',
                      '1 - 0',
                      '1 - 1',
                      '1 - 2',
                      '1 - 3',
                      '1 - 4',
                      '2 - 0',
                      '2 - 1',
                      '2 - 2',
                      '2 - 3',
                      '2 - 4',
                      '3 - 0',
                      '3 - 1',
                      '3 - 2',
                      '3 - 3',
                      '3 - 4',
                      '4 - 0',
                      '4 - 1',
                      '4 - 2',
                      '4 - 3',
                      '4 - 4']
    Pie_Df.loc[Pie_Df["P(score)"] < 0.02, "score"] = 'Other'
    Pie_Df = Pie_Df.sort_values(by=['P(score)'], ascending=False)
    Pie_list.append(Pie_Df)
    

PRED = PRED[['Date', 'HTeam', 'ATeam', 'P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Under)', 'P(Over)', 'P(GG)', 'P(NG)']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(PRED)

pd.options.display.float_format = '{:.2f}'.format
disp_pred = PRED.copy()
disp_pred.rename(columns = {'ATeam':'Away Team'}, inplace = True)
disp_pred.rename(columns = {'HTeam':'Home Team'}, inplace = True)
html_table_blue_light = build_table(disp_pred, 'blue_light')

# Save to html file
with open('UpcomingMatchesPrediction.html', 'w') as f:
    f.write(html_table_blue_light)

Final_Odds = pd.concat([PRED, Odds], axis=0)
Final_Odds['Type'] = np.nan
Final_Odds['Match'] = np.nan
Final_Odds = Final_Odds.reset_index(drop = 'True')
Final_Odds.loc[Final_Odds['B365H'].isnull(), 'Type'] = 'Predicted'
Final_Odds.loc[Final_Odds['P(Hwin)'].isnull(), 'Type'] = 'Bookmaker'

for i, x in enumerate(Final_Odds['Match']):
    Final_Odds['Match'][i] = str(Final_Odds['HTeam'][i] + '-' + Final_Odds['ATeam'][i])

li  = Final_Odds.index[Final_Odds['Type'] == 'Bookmaker'].tolist()
for i in li:
    Final_Odds['P(Hwin)'][i] = 1/Final_Odds.iloc[i]['B365H']
    Final_Odds['P(Awin)'][i] = 1/Final_Odds.iloc[i]['B365A']
    Final_Odds['P(Draw)'][i] = 1/Final_Odds.iloc[i]['B365D']
    Final_Odds['P(Over)'][i] = 1/Final_Odds.iloc[i]['BbAv>2.5']
    Final_Odds['P(Under)'][i] = 1/Final_Odds.iloc[i]['BbAv<2.5']
    Final_Odds['P(GG)'][i] = 1/Final_Odds.iloc[i]['GG']
    Final_Odds['P(NG)'][i] = 1/Final_Odds.iloc[i]['NG']

Final_Odds = Final_Odds[['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Under)', 'P(Over)', 'P(GG)', 'P(NG)', 'Type', 'Match', 'Date']].copy()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(Final_Odds)

    

,Date,HTeam,ATeam,P(Hwin),P(Draw),P(Awin),P(Under),P(Over),P(GG),P(NG)
0,07/10/2023,Luton,Tottenham,0.153,0.189,0.658,0.375,0.625,0.559,0.441
1,07/10/2023,Burnley,Chelsea,0.177,0.227,0.596,0.496,0.504,0.488,0.512
2,07/10/2023,Everton,Bournemouth,0.654,0.200,0.146,0.440,0.560,0.503,0.497
3,07/10/2023,Fulham,Sheffield United,0.607,0.220,0.172,0.476,0.524,0.499,0.501
4,07/10/2023,Manchester United,Brentford,0.512,0.222,0.265,0.370,0.630,0.627,0.373
5,07/10/2023,Crystal Palace,Nottingham Forest,0.464,0.261,0.275,0.532,0.468,0.507,0.493
6,08/10/2023,Wolverhampton Wanderers,Aston Villa,0.169,0.197,0.634,0.378,0.622,0.571,0.429
7,08/10/2023,Brighton,Liverpool,0.472,0.214,0.314,0.296,0.704,0.696,0.304
8,08/10/2023,West Ham,Newcastle United,0.236,0.218,0.546,0.380,0.620,0.608,0.392
9,08/10/2023,Arsenal,Manchester City,0.431,0.222,0.347,0.324,0.676,0.679,0.321


/tmp/ipykernel_12288/2686805503.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['Match'][i] = str(Final_Odds['HTeam'][i] + '-' + Final_Odds['ATeam'][i])
/tmp/ipykernel_12288/2686805503.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['P(Hwin)'][i] = 1/Final_Odds.iloc[i]['B365H']
/tmp/ipykernel_12288/2686805503.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['P(Awin)'][i] = 1/Final_Odds.ilo

,P(Hwin),P(Draw),P(Awin),P(Under),P(Over),P(GG),P(NG),Type,Match,Date
0,0.15,0.19,0.66,0.37,0.63,0.56,0.44,Predicted,Luton-Tottenham,07/10/2023
1,0.18,0.23,0.60,0.50,0.50,0.49,0.51,Predicted,Burnley-Chelsea,07/10/2023
2,0.65,0.20,0.15,0.44,0.56,0.50,0.50,Predicted,Everton-Bournemouth,07/10/2023
3,0.61,0.22,0.17,0.48,0.52,0.50,0.50,Predicted,Fulham-Sheffield United,07/10/2023
4,0.51,0.22,0.27,0.37,0.63,0.63,0.37,Predicted,Manchester United-Brentford,07/10/2023
5,0.46,0.26,0.28,0.53,0.47,0.51,0.49,Predicted,Crystal Palace-Nottingham Forest,07/10/2023
6,0.17,0.20,0.63,0.38,0.62,0.57,0.43,Predicted,Wolverhampton Wanderers-Aston Villa,08/10/2023
7,0.47,0.21,0.31,0.30,0.70,0.70,0.30,Predicted,Brighton-Liverpool,08/10/2023
8,0.24,0.22,0.55,0.38,0.62,0.61,0.39,Predicted,West Ham-Newcastle United,08/10/2023
9,0.43,0.22,0.35,0.32,0.68,0.68,0.32,Predicted,Arsenal-Manchester City,08/10/2023


In [6]:
fig = make_subplots(rows=2, cols=1)
fig.update_layout(width=1500, height=1000)
fig.add_trace(
    go.Bar(name='Predicted Home Win',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Hwin)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Home Win',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Hwin)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Predicted Draw',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Draw)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Draw',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Draw)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Predicted Away Win',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Awin)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Away Win',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Awin)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)


fig.add_trace(
    go.Bar(name='Predicted Over',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Over)'],
           marker_color='rgb(141,211,199)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Over',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Over)'],
           marker_color='rgb(82, 106, 131)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Predicted Under',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Under)'],
           marker_color='rgb(141,211,199)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Under',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Under)'],
           marker_color='rgb(82, 106, 131)'),
    row=2, col=1)


fig.show()


In [7]:
# Create subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}, {'type': 'xy'}]], column_widths=[0.33, 0.67])
fig.update_layout(width=1500, height=650)
# create a list to store the buttons
buttons = []
categories = ['Home', 'Draw', 'Away', 'Over', 'Under', 'GG', 'NG']

# loop through each match and create two traces for Bookmaker odds and predicted values
for match_index, match in enumerate(Final_Odds['Match'].unique().tolist()):

    # create the trace for Bookmaker odds
    trace_bookmaker = go.Scatterpolar(
      r=Final_Odds[(Final_Odds['Type']=='Bookmaker')  & (Final_Odds['Match']==match)][['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Over)', 'P(Under)', 'P(GG)', 'P(NG)']].values.tolist()[0],
      theta=categories,
      fill='toself',
      name='Bookmaker Odds',
      marker = dict(color = 'rgb(82, 106, 131)' ),
      visible=False if match_index!=0 else True,
      hovertemplate='%{theta}<br>Prob: %{r:.2f}<br>Bookmaker Odds'
)
    fig.add_trace(trace_bookmaker, row=1, col=1)
    # create the trace for predicted values
    trace_predicted = go.Scatterpolar(
        r= Final_Odds[(Final_Odds['Type']=='Predicted')  & (Final_Odds['Match']==match)][['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Over)', 'P(Under)', 'P(GG)', 'P(NG)']].values.tolist()[0],
        theta=categories,
        fill='toself',
        name='Predicted Odds',
        marker = dict(color ='rgb(141,211,199)'), #'rgb(217, 175, 107)'
        visible=False if match_index!=0 else True,
        hovertemplate='%{theta}<br>Prob: %{r:.2f}<br>Predicted Odds'
  )
    fig.add_trace(trace_predicted, row=1, col=1)
    
    trace_goals = go.Bar(
        x=Pie_list[match_index]['score'],
        y=Pie_list[match_index]['P(score)'],
        visible=False if match_index!=0 else True,
        marker=dict(color=Pie_list[match_index]['P(score)'], colorscale='darkmint'),
        name='Propable Scoreline',
        hovertemplate='Score %{x}<br>Prob: %{y:.3f}'
    )
    fig.add_trace(trace_goals, row=1, col=2)
    
    # create the dictionary for the button
    button_dict = dict(
        method="restyle",
        args=[{"visible": [False] * (len(Final_Odds['Match'].unique().tolist()) * 3)}], # set all traces to false initially
        label= f"<b>{match} | {str(Final_Odds[(Final_Odds['Type']=='Predicted')  & (Final_Odds['Match']==match)]['Date'].values).strip('[]')}</b>" 
    )
    
    # set visibility to true for the corresponding traces
    button_dict["args"][0]["visible"][3*match_index] = True # Bookmaker odds trace
    button_dict["args"][0]["visible"][3*match_index+1] = True # Predicted trace
    button_dict["args"][0]["visible"][3*match_index+2] = True # Goal trace
    
    # set visibility to true for the corresponding traces for the first match
    if match_index == 0:
        button_dict["args"][0]["visible"][0] = True # Bookmaker odds trace
        button_dict["args"][0]["visible"][1] = True # Predicted trace
        button_dict["args"][0]["visible"][2] = True
        
    # append the button to the list
    buttons.append(button_dict)


# update the layout with the buttons
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        buttons=buttons,
        active=0, # set the initial active button
        x=0.1,
        y=1.2,
        direction="down",
    )],
    
)

# show the plot
fig.show()
# Save the interactive figure as an HTML file
fig.write_html('UpcomingMatchesPredictionFigure.html')


In [8]:
Pont = 5
bank = 60
Final = Final_Odds[Final_Odds['Type']=='Predicted'][['Match']]
listPred = Final_Odds[Final_Odds['Type']=='Predicted']['P(Under)'].tolist()
listBook = Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Under)'].tolist()

diff = []
for i, p in enumerate(listPred):
    diff.append(listPred[i]-listBook[i])

Final['UnderBet'] = np.nan
Final['UnderBet_Kelly'] = np.nan
for i, d in enumerate(diff):
    if (0.7>=listPred[i]>=0.5) and (d>=0.005):
        Final['UnderBet'][i] = max(Pont*10*(d),0)
        Final['UnderBet_Kelly'][i] = max(0, bank*(listPred[i]- (1-listPred[i])/(1/listBook[i] -1)))
    else:
        Final['UnderBet'][i] = 0
        Final['UnderBet_Kelly'][i] = 0
        
############

    
display(Final)

/tmp/ipykernel_12288/1283783480.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_12288/1283783480.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Match,UnderBet,UnderBet_Kelly
0,Luton-Tottenham,0.00,0.00
1,Burnley-Chelsea,0.00,0.00
2,Everton-Bournemouth,0.00,0.00
3,Fulham-Sheffield United,0.00,0.00
4,Manchester United-Brentford,0.00,0.00
5,Crystal Palace-Nottingham Forest,0.00,0.00
6,Wolverhampton Wanderers-Aston Villa,0.00,0.00
7,Brighton-Liverpool,0.00,0.00
8,West Ham-Newcastle United,0.00,0.00
9,Arsenal-Manchester City,0.00,0.00
